## Create graphs for streamlines

In [23]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this Jupyter notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">HERE</a>.''')

# By Damian Kao; http://blog.nextgenetics.net/?e=102

In [ ]:
import plotly
if plotly.__version__ < '4.5.1':
    if 'google.colab' in str(get_ipython()):
        print('Updating Plotly library')
        !pip install plotly==4.5.1
    else:
        print("Version '4.5.1' is necessary; try:\n\t !pip install plotly==4.5.1")

### Modules

In [1]:
from os.path import join
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mc
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

### Load data

In [106]:
if 'google.colab' in str(get_ipython()): # if running in colab
    !curl -L -o drop_files.zip 'https://www.dropbox.com/sh/vdslcrx0vg0sm39/AABBVRgKO6P71ip3DnPoCpBEa'
    !unzip -q drop_files.zip -d maps_values_dir
    !echo "   Data downloaded in: 'maps_values_dir/'"
    map_folder = "maps_values_dir"
    all_rats_file = join(map_folder, 'rat_dataset.csv')
else:
    # from Local, run
    all_rats_file = '/data/Documentos/Maestría/LabC13/Rata_Data/rat_dataset.csv'
    # folder with mapped streamlines files (.txt)
    map_folder = "/data/Documentos/Dropbox/INB/C13_open_data"

In [36]:
# load data (run always)
all_rats = pd.read_csv(all_rats_file)
# control & bcnu idx list
all_control_id = all_rats[all_rats['group']=='Control'].rat_id.to_list()
all_bcnu_id = all_rats[all_rats['group']=='BCNU'].rat_id.to_list()
all_rats.head()

,rat_id,group,sex,slice_n
0,37A,Control,Male,14
1,37C,Control,Male,12
2,37D,Control,Male,13
3,38F,Control,Female,12
4,39A,BCNU,Male,12


### Load variables

In [41]:
# create dictionary with stream map values
def dict_from_map(metric_map='dti_fa'):
    """
    metric_map : str= dti_ad dti_adc dti_rd dti_fa
                    intra filt icvf cortex_long
    """
    metric_dict = {}
    for rat_id in all_control_id + all_bcnu_id: 
        slice_n = all_rats.slice_n[all_rats.rat_id==rat_id].values[0]
        txt_file = f"{rat_id}_l_{slice_n}_{metric_map}.txt"
        try:
            metric_dict[rat_id] = np.loadtxt(join(map_folder, txt_file))
        except OSError:
            print('File', join(map_folder, txt_file), ' not found')
    return metric_dict


def dict_values(metric_dict):
    """
    returns
        2 arrays (control & bcnu)
    """
    control_id = [val for val in list(metric_dict.keys()) if val in all_control_id]
    bcnu_id = [val for val in list(metric_dict.keys()) if val in all_bcnu_id]

    control_values = np.empty([len(control_id), 150, 20])
    for i, rat_id in enumerate(control_id):
        control_values[i] = metric_dict[rat_id]
    bcnu_values = np.empty([len(bcnu_id), 150, 20])
    for i, rat_id in enumerate(bcnu_id):
        bcnu_values[i] = metric_dict[rat_id]
    print(f"\n\t{len(control_id)} control subjects:", *control_id)
    print(f"\t{len(bcnu_id)} bcnu subjects:", *bcnu_id)
    return control_values, bcnu_values
        
fa_dict = dict_from_map(metric_map='dti_fa')

control_id = [val for val in list(fa_dict.keys()) if val in all_control_id]
bcnu_id = [val for val in list(fa_dict.keys()) if val in all_bcnu_id]

control_values = np.empty([len(control_id), 150, 20])
for i, rat_id in enumerate(control_id):
    control_values[i] = fa_dict[rat_id]
bcnu_values = np.empty([len(bcnu_id), 150, 20])
for i, rat_id in enumerate(bcnu_id):
    bcnu_values[i] = fa_dict[rat_id]
print(f"\n\t{len(control_id)} control subjects:", *control_id)
print(f"\t{len(bcnu_id)} bcnu subjects:", *bcnu_id)


File /data/Documentos/Dropbox/INB/C13_open_data/48I_l_11_dti_fa.txt  not found
File /data/Documentos/Dropbox/INB/C13_open_data/47H_l_11_dti_fa.txt  not found

	19 control subjects: 37A 37C 37D 38F 40A 40D 46A 46B 46E 46G 46H 46I 46J 48A 48G 48H 52A 56B 57B
	18 bcnu subjects: 39A 39B 42A 42F 47A 47B 47C 47D 47E 47I 47J 47K 49G 49H 49I 54D 54E 55A


In [5]:
# OLD data
#rata_data="/home/dcortex/Documentos/Maestría/LabC13/Rata_Data"
#join(rata_data,'37A','stats')
#stats = []
#stats.append(np.loadtxt(join(rata_data,'37A','stats','37A_l_14_dti_fa.txt')))
#fa = stats[0]
#fa_m = np.mean(fa, axis=0)
#x = np.arange(20)

### Functions

In [25]:
def plot_streams(streams, fig=None, title='', streams_2=None, y_title='FA', CI=False,
                 style='plotly_dark', labels=None, labels_2=None, legend_singles=True,
                 legend_title='Streams', y_range=[0,1], plot_singles=True, row=None, col=None,
                 mean_color="aqua", ci_color="aqua", mean_label='&#956;', ci_label='2&#963;',
                 mean_label_2='&#956; BCNU', ci_label_2='2&#963; BCNU', singles_color=None,
                 legendgroup=None,
                 ):
    """
    Parameters
        streams : array_like (streams x depth)
    Returns
        fig : plotly.graph_objs._figure.Figure
    """
    if fig is None:
        fig = go.Figure()
        fig.update_layout(title=title, ) 
    fig.layout.template = style
    fig.update_layout(legend_title=legend_title, ) 
    n, k = streams.shape
    x = np.arange(k)
    if labels is not None and len(labels)==n:
        names = labels
    else:
        names = np.arange(n)
    if plot_singles:
        if singles_color is None:
            colors = plt.get_cmap('jet_r', n)
        else: # same color for all streams
            c = mc.to_hex(singles_color)
        if legendgroup is not None:
            legend_singles = [True] + [False]*(len(streams)-1) # only first legend shows
            names = [legendgroup]*len(streams)
        else: legend_singles = [legend_singles]*len(streams)
        for i, s in enumerate(streams): #            <--- singles plot
            if singles_color is None:
                c = mc.to_hex(colors(i))
            fig.add_trace(go.Line(x=x, y=s, name=str(names[i]), line=dict(width=1, color=c),
                                  opacity=1, showlegend=legend_singles[i], legendgroup=legendgroup,
                                  ),
                          row=row, col=col)
    mean = np.mean(streams, axis=0)
    fig.add_trace(go.Line(y=mean, name=mean_label, opacity=0.25, # <--- mean plot
                          line=dict(color=mean_color, width=6, dash='dash')),
                  row=row, col=col) 
    if CI:
        std_dev = np.std(streams, axis=0, ddof=1)
        a = mean+2*std_dev
        b = mean-2*std_dev
        fig.add_trace(go.Scatter(x=x, y=a, hoverinfo='none',# upper
                                 line=dict(width=0), showlegend=False ), row=row, col=col, )
        c = 'rgba'+ str(mc.to_rgba(ci_color,alpha=0.2))
        fig.add_trace(go.Scatter(x=x, y=b, name=ci_label, #  lower 
                                 line=dict(width=0), showlegend=True,
                                 fillcolor=c,
                                 fill='tonexty'),  row=row, col=col,)
    if streams_2 is not None:
        if legendgroup is not None: legendgroup_2="BCNUs"
        else: legendgroup_2=legendgroup
        plot_streams(streams=streams_2, fig=fig, CI=CI, labels=labels_2, # <--- second array to plot
                     legend_singles=legend_singles[0], y_range=y_range,
                     plot_singles=plot_singles, row=row, col=col,
                     mean_color="darkorange", ci_color="darkorange",
                     mean_label=mean_label_2, ci_label=ci_label_2,
                     singles_color='lightsalmon', legendgroup=legendgroup_2,
                     )
        
    fig.update_yaxes(title_text=y_title, row=row, col=col, range=y_range,
                     ticks="outside", nticks = 10, title_standoff = 0, )
    fig.update_xaxes(title_text='Cortical depth (normalized)', row=row, col=col,
                     ticks="outside", tickvals = [0,19],
                     ticktext = ['Outer<br />cortex', 'Inner<br />cortex'],
                     title_standoff = 0, )    
    #fig.layout.template = style
    # 'ggplot2', 'seaborn', 'simple_white',
    # 'plotly', 'plotly_white', 'plotly_dark', 'none'
    # 'presentation', 'xgridoff', 'ygridoff', 'gridon'
    return fig

In [31]:
def abnormality_load(control_streams, bcnu_streams, fig=None, title='', style='ggplot2',
                     legend_title='', row=None, col=None,
                     ):          
    """
     Returns
        ab_load : array with percent of stream outliers by depth (normalized)
    """
    mean = np.mean(control_streams, axis=0)
    std_dev = np.std(control_streams, axis=0, ddof=1)
    a = mean+2*std_dev
    b = mean-2*std_dev
    
    ab_load = np.zeros(20)
    for y in bcnu_streams:
        aux = np.logical_and(b<y,y<a)
        ab_load += np.logical_not(aux)
    ab_load = (ab_load*100)/(len(bcnu_streams))
        
    return ab_load
    #return np.mean(ab_load)
    
def plot_abnormality(ab_load, fig=None, title='', style='plotly_dark', legend_title='',
                     x_title='Cortical depth (normalized)', y_title='% stream outliers',
                     x_ticktext=['Outer<br />cortex','Inner<br />cortex'], row=None, col=None,
                     plot_color='crimson',
                     ):
    """
    """
    if fig is None:
        fig = go.Figure()
        fig.update_layout(title=title, hovermode='x')
    fig.layout.template = style
    fig.update_layout(legend_title=legend_title, )
    
    fig.add_trace(go.Line(y=ab_load, showlegend=False, line=dict(color=plot_color),
                          #name=str(names[i]), opacity=1, showlegend=legend_singles),
                          ), row=row, col=col,)
    
    fig.update_yaxes(title_text=y_title, row=row, col=col, #range=[0,200],
                     ticks="outside", nticks = 10, range=[0-max(ab_load)*0.1, max(ab_load)*1.1],
                     title_standoff = 0,
                     )
    fig.update_xaxes(title_text=x_title, row=row, col=col,
                     ticks="outside", tickvals = [0,len(ab_load)-1], # range=[0,19],
                     ticktext=x_ticktext, title_standoff = 0, ) 
    
    return fig

In [26]:
# Save fig as html file:
#fig1.write_html("/data/Descargas/file.html")

In [27]:
# test plot_streams
stream = 127
X, Y = control_values[:,stream,:],  bcnu_values[:,stream,:]
f = plot_streams(X, fig=None, title=f"FA values, stream {stream}, Control vs BCNU",
                 mean_label='&#956; Control', ci_label='2&#963; Control',
                 CI=True, legend_singles=True, plot_singles=True, streams_2=Y,
                 singles_color='royalblue',
                 labels=control_id, labels_2=bcnu_id,
                 legendgroup='Controls', # group 
                 )

/home/dcortex/anaconda3/lib/python3.7/site-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [28]:
f

In [30]:
#f

In [32]:
# Abnormality 
all_ab_load_sum = []
all_ab_load_mean = []
for s in range(150):
    X = control_values[:,s,:]
    Y = bcnu_values[:,s,:]
    ab_load = abnormality_load(X, Y)
    all_ab_load_sum.append(sum(ab_load))
    all_ab_load_mean.append(np.mean(ab_load))

In [33]:
fig_ab=plot_abnormality(np.array(all_ab_load_mean),
                        title='Mean (along depth) abnormality load per streamline',
                        x_title='Stream', y_title='&#956;(% streams)',
                        x_ticktext=['Medial<br />cortex', 'Lateral<br />cortex'],
                        )

In [34]:
fig_ab.show()

In [13]:
"""
id_1 = '37A'
id_2 = '37A'
X = fa_dict[id_1]
Y = fa_dict[id_2]
control_streams, bcnu_streams = X, Y
mean = np.mean(control_streams, axis=0)
std_dev = np.std(control_streams, axis=0, ddof=1)
a = mean+2*std_dev
b = mean-2*std_dev
"""

"\nid_1 = '37A'\nid_2 = '37A'\nX = fa_dict[id_1]\nY = fa_dict[id_2]\ncontrol_streams, bcnu_streams = X, Y\nmean = np.mean(control_streams, axis=0)\nstd_dev = np.std(control_streams, axis=0, ddof=1)\na = mean+2*std_dev\nb = mean-2*std_dev\n"

In [14]:
# Plot 2 figures: streamlines by group & abnormality load:
stream = 127
fig1 = make_subplots(rows=1, cols=2,
                     subplot_titles=['Abnormality load', 'FA values'],
                     horizontal_spacing=0.1,
                     )
fig1.update_layout(title_text=f"Stream {stream}", title_x=0.5,
                   margin=go.layout.Margin(t=50,b=50 ), )
# Streams plot
X, Y = control_values[:,stream,:],  bcnu_values[:,stream,:]
f = plot_streams(X,  fig=fig1, title=f"FA values, stream {stream}, Control vs BCNU",
                 CI=True, legend_singles=True, plot_singles=True,
                 streams_2=Y, mean_label='&#956; Control', ci_label='2&#963; Control',
                 row=1, col=2, singles_color='royalblue', 
                 )
# Abnormality plot
ab_load_single = abnormality_load(X, Y)
f_ab = plot_abnormality(ab_load_single, fig=fig1,
                        title=f'Abnormality load for streamline={stream}',
                        y_title='% streams',
                        row=1, col=1,
                        )

In [15]:
fig1

### ICVF

In [44]:
icvf_dict = dict_from_map(metric_map='icvf')
# arrays
icvf_control_values, icvf_bcnu_values = dict_values(metric_dict=icvf_dict)

File /data/Documentos/Dropbox/INB/C13_open_data/48I_l_11_icvf.txt  not found
File /data/Documentos/Dropbox/INB/C13_open_data/52A_l_13_icvf.txt  not found
File /data/Documentos/Dropbox/INB/C13_open_data/56B_l_13_icvf.txt  not found
File /data/Documentos/Dropbox/INB/C13_open_data/57B_l_12_icvf.txt  not found
File /data/Documentos/Dropbox/INB/C13_open_data/47H_l_11_icvf.txt  not found
File /data/Documentos/Dropbox/INB/C13_open_data/49I_l_12_icvf.txt  not found
File /data/Documentos/Dropbox/INB/C13_open_data/54D_l_12_icvf.txt  not found
File /data/Documentos/Dropbox/INB/C13_open_data/54E_l_12_icvf.txt  not found
File /data/Documentos/Dropbox/INB/C13_open_data/55A_l_12_icvf.txt  not found

	16 control subjects: 37A 37C 37D 38F 40A 40D 46A 46B 46E 46G 46H 46I 46J 48A 48G 48H
	14 bcnu subjects: 39A 39B 42A 42F 47A 47B 47C 47D 47E 47I 47J 47K 49G 49H


In [78]:
def plot_stream_2_graphs(control_values, bcnu_values, stream=127, metric='FA', y_range=[0,1]):
    X, Y = control_values[:,stream,:],  bcnu_values[:,stream,:]

    fig1 = make_subplots(rows=1, cols=2,
                         subplot_titles=['Abnormality load', metric+' values'],
                         horizontal_spacing=0.1,
                         )
    fig1.update_layout(title_text=f"Stream {stream}", title_x=0.5,
                       margin=go.layout.Margin(t=50,b=50 ), )
    # Streams plot
    plot_streams(X,  fig=fig1, title=f"{metric} values, stream {stream}, Control vs BCNU",
                 CI=True, legend_singles=True, plot_singles=True,
                 streams_2=Y, mean_label='&#956; Control', ci_label='2&#963; Control',
                 row=1, col=2, singles_color='royalblue', y_range=y_range, 
                 )
    # Abnormality plot
    ab_load_single = abnormality_load(X, Y)
    plot_abnormality(ab_load_single, fig=fig1,
                     title=f'Abnormality load for streamline={stream}',
                     y_title='% streams', row=1, col=1,
                     )
    return fig1

    # Abnormality
def plot_abnormality_all(control_values, bcnu_values, metric='FA'):
    all_ab_load_mean = []
    for s in range(150):
        X = control_values[:,s,:]
        Y = bcnu_values[:,s,:]
        ab_load = abnormality_load(X, Y)
        all_ab_load_mean.append(np.mean(ab_load))
    fig = plot_abnormality(np.array(all_ab_load_mean),
                        title='Mean (along depth) abnormality load per streamline for '+metric,
                        x_title='Stream', y_title='&#956;(% streams)',
                        x_ticktext=['Medial<br />cortex', 'Lateral<br />cortex'],
                        )
    return fig

In [57]:
f1 = plot_stream_2_graphs(icvf_control_values, icvf_bcnu_values, stream=17, metric='ICVF')
f2 = plot_stream_2_graphs(icvf_control_values, icvf_bcnu_values, stream=100, metric='ICVF')
f3 = plot_stream_2_graphs(icvf_control_values, icvf_bcnu_values, stream=138, metric='ICVF')
f1.show()
f2.show()
f3.show()

In [63]:
f4 = plot_abnormality_all(icvf_control_values, icvf_bcnu_values, metric='ICVF')
f4.show()

In [64]:
f1.write_html("/data/Descargas/icvf_s17.html")
f2.write_html("/data/Descargas/icvf_s100.html")
f3.write_html("/data/Descargas/icvf_s138.html")
f4.write_html("/data/Descargas/icvf_ab_total.html")

### INTRA

In [67]:
intra_dict = dict_from_map(metric_map='intra')
# arrays
intra_control_values, intra_bcnu_values = dict_values(metric_dict=intra_dict)

File /data/Documentos/Dropbox/INB/C13_open_data/48I_l_11_intra.txt  not found
File /data/Documentos/Dropbox/INB/C13_open_data/52A_l_13_intra.txt  not found
File /data/Documentos/Dropbox/INB/C13_open_data/56B_l_13_intra.txt  not found
File /data/Documentos/Dropbox/INB/C13_open_data/57B_l_12_intra.txt  not found
File /data/Documentos/Dropbox/INB/C13_open_data/47H_l_11_intra.txt  not found
File /data/Documentos/Dropbox/INB/C13_open_data/49I_l_12_intra.txt  not found
File /data/Documentos/Dropbox/INB/C13_open_data/54D_l_12_intra.txt  not found
File /data/Documentos/Dropbox/INB/C13_open_data/54E_l_12_intra.txt  not found
File /data/Documentos/Dropbox/INB/C13_open_data/55A_l_12_intra.txt  not found

	16 control subjects: 37A 37C 37D 38F 40A 40D 46A 46B 46E 46G 46H 46I 46J 48A 48G 48H
	14 bcnu subjects: 39A 39B 42A 42F 47A 47B 47C 47D 47E 47I 47J 47K 49G 49H


In [70]:
f8 = plot_abnormality_all(intra_control_values, intra_bcnu_values, metric='INTRA')
f8.show()

In [71]:
f5 = plot_stream_2_graphs(intra_control_values, intra_bcnu_values, stream=35, metric='INTRA')
f6 = plot_stream_2_graphs(intra_control_values, intra_bcnu_values, stream=100, metric='INTRA')
f7 = plot_stream_2_graphs(intra_control_values, intra_bcnu_values, stream=117, metric='INTRA')
f5.show()
f6.show()
f7.show()

In [72]:
f5.write_html("/data/Descargas/intra_s35.html")
f6.write_html("/data/Descargas/intra_s100.html")
f7.write_html("/data/Descargas/intra_s117.html")
f8.write_html("/data/Descargas/intra_ab_total.html")

### FILT

In [74]:
filt_dict = dict_from_map(metric_map='filt')
# arrays
filt_control_values, filt_bcnu_values = dict_values(metric_dict=filt_dict)

File /data/Documentos/Dropbox/INB/C13_open_data/48I_l_11_filt.txt  not found
File /data/Documentos/Dropbox/INB/C13_open_data/52A_l_13_filt.txt  not found
File /data/Documentos/Dropbox/INB/C13_open_data/56B_l_13_filt.txt  not found
File /data/Documentos/Dropbox/INB/C13_open_data/57B_l_12_filt.txt  not found
File /data/Documentos/Dropbox/INB/C13_open_data/47H_l_11_filt.txt  not found
File /data/Documentos/Dropbox/INB/C13_open_data/49I_l_12_filt.txt  not found
File /data/Documentos/Dropbox/INB/C13_open_data/54D_l_12_filt.txt  not found
File /data/Documentos/Dropbox/INB/C13_open_data/54E_l_12_filt.txt  not found
File /data/Documentos/Dropbox/INB/C13_open_data/55A_l_12_filt.txt  not found

	16 control subjects: 37A 37C 37D 38F 40A 40D 46A 46B 46E 46G 46H 46I 46J 48A 48G 48H
	14 bcnu subjects: 39A 39B 42A 42F 47A 47B 47C 47D 47E 47I 47J 47K 49G 49H


In [76]:
f12 = plot_abnormality_all(filt_control_values, filt_bcnu_values, metric='FILT')
f12.show()

In [86]:
y_r = [-0.8e-9, 2.5e-9]
f9 = plot_stream_2_graphs(filt_control_values, filt_bcnu_values, stream=35, metric='FILT', y_range=y_r)
f10 = plot_stream_2_graphs(filt_control_values, filt_bcnu_values, stream=100, metric='FILT', y_range=y_r)
f11 = plot_stream_2_graphs(filt_control_values, filt_bcnu_values, stream=138, metric='FILT', y_range=y_r)
f9.show()
f10.show()
f11.show()

In [87]:
f9.write_html("/data/Descargas/filt_s35.html")
f10.write_html("/data/Descargas/filt_s100.html")
f11.write_html("/data/Descargas/filt_s138.html")
f12.write_html("/data/Descargas/filt_ab_total.html")

### cortex_long

In [75]:
cx_long_dict = dict_from_map(metric_map='cortex_long')
# arrays
cx_long_control_values, cx_long_bcnu_values = dict_values(metric_dict=cx_long_dict)

File /data/Documentos/Dropbox/INB/C13_open_data/48I_l_11_cortex_long.txt  not found
File /data/Documentos/Dropbox/INB/C13_open_data/52A_l_13_cortex_long.txt  not found
File /data/Documentos/Dropbox/INB/C13_open_data/56B_l_13_cortex_long.txt  not found
File /data/Documentos/Dropbox/INB/C13_open_data/57B_l_12_cortex_long.txt  not found
File /data/Documentos/Dropbox/INB/C13_open_data/47H_l_11_cortex_long.txt  not found
File /data/Documentos/Dropbox/INB/C13_open_data/49I_l_12_cortex_long.txt  not found
File /data/Documentos/Dropbox/INB/C13_open_data/54D_l_12_cortex_long.txt  not found
File /data/Documentos/Dropbox/INB/C13_open_data/54E_l_12_cortex_long.txt  not found
File /data/Documentos/Dropbox/INB/C13_open_data/55A_l_12_cortex_long.txt  not found

	16 control subjects: 37A 37C 37D 38F 40A 40D 46A 46B 46E 46G 46H 46I 46J 48A 48G 48H
	14 bcnu subjects: 39A 39B 42A 42F 47A 47B 47C 47D 47E 47I 47J 47K 49G 49H


In [88]:
f16 = plot_abnormality_all(cx_long_control_values, cx_long_bcnu_values, metric='CX_long')
f16.show()

In [93]:
y_r = [-2e-3,5e-3]
f13 = plot_stream_2_graphs(cx_long_control_values, cx_long_bcnu_values, stream=35, metric='CX_long', y_range=y_r)
f14 = plot_stream_2_graphs(cx_long_control_values, cx_long_bcnu_values, stream=100, metric='CX_long', y_range=y_r)
f15 = plot_stream_2_graphs(cx_long_control_values, cx_long_bcnu_values, stream=138, metric='CX_long', y_range=y_r)
f13.show()
f14.show()
f15.show()

In [94]:
f13.write_html("/data/Descargas/cx_long_s35.html")
f14.write_html("/data/Descargas/cx_long_s100.html")
f15.write_html("/data/Descargas/cx_long_s138.html")
f16.write_html("/data/Descargas/cx_long_ab_total.html")

In [95]:
plotly.__version__

'4.5.1'

In [96]:
"""
!pip install plotly==4.5.1
"""

'\n!pip install plotly==4.5.1\n'